### DNN-Keras


In [8]:
# importing the packages
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import date
import sklearn.exceptions
from pylab import rcParams
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.svm import OneClassSVM
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import IsolationForest
from imblearn.under_sampling import NearMiss 
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)


from numpy import asarray
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

In [9]:

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import os
import matplotlib.pyplot as plt
import itertools
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, f1_score, recall_score

## Data Preprocessing steps

In [10]:
data_train = pd.read_csv("fraudTrain.csv")
data_test = pd.read_csv("fraudTest.csv")
# taking smaller sample to run the model faster

df_train= data_train.sample(frac = 0.1,random_state=1)
df_test= data_test.sample(frac = 0.05,random_state=1)
print(df_train.shape,df_test.shape)

(129668, 23) (27786, 23)


In [11]:
def getdatafiles():
    # loading data
    data_train = pd.read_csv("fraudTrain.csv")
    data_test = pd.read_csv("fraudTest.csv")
    # taking smaller sample to run the model faster

    df_train= data_train.sample(frac = 0.1,random_state=1)
    df_test= data_test.sample(frac = 0.05,random_state=1)
    print(df_train.shape,df_test.shape)
    
    # function to drop tbe columns
    def dropCol(data):
        col_to_drop = ['trans_date_trans_time','Unnamed: 0',
                       'cc_num','first','last','trans_num','street','unix_time']
        res = data.drop(col_to_drop,axis = 1)
        return res
    # dropping the columns
    # dropping the columns ['trans_date_trans_time','Unnamed: 0','cc_num','first','last','trans_num']
    # train data set
    df_train = dropCol(df_train)
    # test data set
    df_test = dropCol(df_test)
    
#     # creating a new feature address
#     df_train['add'] = df_train['city'] + df_train['state'] + df_train['zip'].astype(str)
#     df_test['add'] = df_test['city'] + df_test['state'] + df_test['zip'].astype(str)
#     df_train = df_train.drop(['city','state','zip'],axis =1)
#     df_test = df_test.drop(['city','state','zip'],axis =1)
#     print ( df_train.shape, df_test.shape)
    

    #Create independent and Dependent Features
    columns = df_train.columns.tolist()

    # removing the dependent feature is_fraud
    columns = [c for c in columns if c not in ["is_fraud"]]

    X_train = df_train[columns]
    Y_train = df_train['is_fraud']
    X_test = df_test[columns]
    Y_test = df_test['is_fraud']
    print ( X_train.shape, Y_train.shape,X_test.shape, Y_test.shape)

    # function to convert dob to years
    def age_years(born):
        return 2019 - int(born[0:4])

    # replacing the dob column with age column in our data set for test and train
    X_train['age'] = X_train['dob'].apply(lambda x: age_years(x))
    X_train = X_train.drop(['dob'],axis =1)

    X_test['age'] = X_test['dob'].apply(lambda x: age_years(x))
    X_test = X_test.drop(['dob'],axis =1)
    
    print(X_train.shape,X_test.shape)

    # concanating the test and train data so that number of columns remain the same in both the data sets
    final_df = pd.concat([X_train,X_test],axis=0)
    final_df.shape


    # creating the list of categorical variables
    categorical_features =[feature for feature in X_train.columns if final_df[feature].dtypes == 'O']
    categorical_features
    
    # function to convert categorical variables to one hot encoding
    def category_onehot_multcols(data,multcolumns):
        df_final = data
        i=0
        for fields in multcolumns:
            print(fields)
            df1=pd.get_dummies(final_df[fields],drop_first=True)
            final_df.drop([fields],axis=1,inplace=True)
            if i==0:
                df_final=df1.copy()
            else:           
                df_final=pd.concat([df_final,df1],axis=1)
            i=i+1             
        df_final=pd.concat([final_df,df_final],axis=1)
        return df_final

    # applying the one hot encoding
    final_df = category_onehot_multcols(final_df, categorical_features)

    # removing duplicated columns
    final_df =final_df.loc[:,~final_df.columns.duplicated()]
    final_df.shape

    # separating the test and training data
    df_Train=final_df.iloc[:129668,:]
    df_Test=final_df.iloc[129668:,:]
    print(df_Train.shape,df_Test.shape)
    print(Y_train.value_counts(),"\n",Y_test.value_counts())
    
    # define standard scaler
    scaler = StandardScaler()
    # transform data
    df_Train = scaler.fit_transform(df_Train)
    df_Test = scaler.fit_transform(df_Test)# define standard scaler
    
    
    
    # files ready for testing on models
    print(df_Train.shape, df_Test.shape, Y_train.shape, Y_test.shape)
    return df_Train, df_Test, Y_train, Y_test


In [12]:
# function to evaluate our model 
def print_eval(y_pred,model):
    print("Training Accuracy: ",model.score(df_Train, Y_train))
    print("Testing Accuracy: ", model.score(df_Test, Y_test))
    print(confusion_matrix(Y_test, y_pred))
    
    print(classification_report(Y_test,y_pred))


In [13]:
df_Train, df_Test, Y_train, Y_test = getdatafiles()

(129668, 23) (27786, 23)
(129668, 14) (129668,) (27786, 14) (27786,)
(129668, 14) (27786, 14)
merchant
category
gender
city
state
job
(129668, 2120) (27786, 2120)
0    128963
1       705
Name: is_fraud, dtype: int64 
 0    27656
1      130
Name: is_fraud, dtype: int64
(129668, 2120) (27786, 2120) (129668,) (27786,)


In [14]:
print(df_Train.shape)

(129668, 2120)


In [57]:
model = Sequential()
#add input layer
model.add(Dense(input_dim = 2120, units = 16, activation = 'relu'))
#add 2nd hidden layer
model.add(Dense(units = 50, activation = 'relu'))
#add dropout layer
#model.add(Dropout(0.5))
#add 3rd hidden layer
model.add(Dense(units = 20, activation = 'relu'))
#add 4th hidden layer
model.add(Dense(units = 50, activation = 'relu'))
#add ouptut layer
model.add(Dense(units = 1, activation = 'sigmoid'))

In [58]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_30 (Dense)             (None, 16)                33936     
_________________________________________________________________
dense_31 (Dense)             (None, 50)                850       
_________________________________________________________________
dense_32 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_33 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 51        
Total params: 36,907
Trainable params: 36,907
Non-trainable params: 0
_________________________________________________________________


In [59]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.fit(df_Train, Y_train, batch_size = 128, epochs = 150)

Epoch 1/150
1014/1014 [==============================] - 3s 2ms/step - loss: 0.0919 - accuracy: 0.9804
Epoch 2/150
1014/1014 [==============================] - 1s 1ms/step - loss: 0.0185 - accuracy: 0.9947
Epoch 3/150
1014/1014 [==============================] - 1s 1ms/step - loss: 0.0123 - accuracy: 0.9958
Epoch 4/150
1014/1014 [==============================] - 1s 1ms/step - loss: 0.0079 - accuracy: 0.9972
Epoch 5/150
1014/1014 [==============================] - 1s 1ms/step - loss: 0.0064 - accuracy: 0.9976
Epoch 6/150
1014/1014 [==============================] - 1s 1000us/step - loss: 0.0044 - accuracy: 0.9983
Epoch 7/150
1014/1014 [==============================] - 1s 1ms/step - loss: 0.0037 - accuracy: 0.9984
Epoch 8/150
1014/1014 [==============================] - 1s 999us/step - loss: 0.0033 - accuracy: 0.9989
Epoch 9/150
1014/1014 [==============================] - 1s 1000us/step - loss: 0.0023 - accuracy: 0.9991
Epoch 10/150
1014/1014 [==============================] - 1s 1ms/

1014/1014 [==============================] - 1s 1ms/step - loss: 5.9267e-04 - accuracy: 0.9997
Epoch 78/150
1014/1014 [==============================] - 1s 1ms/step - loss: 7.2288e-04 - accuracy: 0.9997
Epoch 79/150
1014/1014 [==============================] - 1s 1ms/step - loss: 5.7966e-04 - accuracy: 0.9997
Epoch 80/150
1014/1014 [==============================] - 1s 1ms/step - loss: 6.2118e-04 - accuracy: 0.9997
Epoch 81/150
1014/1014 [==============================] - 1s 1ms/step - loss: 5.3875e-04 - accuracy: 0.9997
Epoch 82/150
1014/1014 [==============================] - 1s 1ms/step - loss: 5.3468e-04 - accuracy: 0.9997
Epoch 83/150
1014/1014 [==============================] - 1s 1ms/step - loss: 5.4151e-04 - accuracy: 0.9996
Epoch 84/150
1014/1014 [==============================] - 1s 1ms/step - loss: 4.7797e-04 - accuracy: 0.9997
Epoch 85/150
1014/1014 [==============================] - 1s 1ms/step - loss: 5.7075e-04 - accuracy: 0.9997
Epoch 86/150
1014/1014 [=================

In [60]:
score = model.evaluate(df_Test, Y_test)
print(score)

869/869 [==============================] - 1s 585us/step - loss: 0.1671 - accuracy: 0.9910
[0.16711823642253876, 0.9910386800765991]


In [61]:

y_pred = model.predict(df_Test)

In [62]:
cm = confusion_matrix(Y_test, y_pred.round())
print(cm)

[[27503   153]
 [   96    34]]


In [63]:
print(accuracy_score(Y_test, y_pred.round()))
print(precision_score(Y_test, y_pred.round()))
print(recall_score(Y_test, y_pred.round()))
print(f1_score(Y_test, y_pred.round()))

0.9910386525588426
0.18181818181818182
0.26153846153846155
0.21451104100946375
